## Setup

Necessary imports, variable and function definitions.

__Note:__ Execution of the sequence-based part of this notebook requires a very large amount of disk space (>1TB).

In [ ]:
import os

import matplotlib.pyplot as plt
import skbio
import pandas as pd
import qiime2 as q2
import seaborn as sns

from qiime2.plugins import (
    fondue, coordinates, demux, sourmash, 
    cutadapt, diversity, emperor
)

%matplotlib inline

results_path = 'u3-metagenome-results'
if not os.path.isdir(results_path):
    os.mkdir(results_path)

In [ ]:
email = 'your@email.com'
n_jobs = 8
random_seed = 1

marine_meta_path = os.path.join(results_path, 'tara_metadata.tsv')
marine_meta_clean_path = os.path.join(results_path, 'tara_metadata_clean.tsv')

In [ ]:
def clean_up_column(df, col, max_value):
    """
    Cleans up specified DataFrame column by converting it
    to a numeric type and removing values beyond the
    indicated maximum.
    """
    if isinstance(df.iloc[0][col], str):
        df = df[~marine_meta_df[col].str.contains('-')]
    df[col] = pd.to_numeric(df[col])
    df = df[df[col] < max_value]
    return df

## Fetch TARA Oceans shotgun metagenome metadata

We will fetch metadata for all the **shotgun sequencing experiments** for size fractionated samples taken all over the world (including the Polar Circle). For overview of BioProject IDs see the [respective entry on ENA](https://www.ebi.ac.uk/ena/browser/view/PRJEB402).

In [ ]:
ids = ['PRJEB1787', 'PRJEB4352', 'PRJEB4419', 'PRJEB9691', 'PRJEB9740', 'PRJEB9742']
ids = pd.Series(ids, name='ID')
ids = q2.Artifact.import_data('NCBIAccessionIDs', ids)

In [ ]:
if not os.path.isfile(marine_meta_path):
    marine_meta, _,  = fondue.methods.get_metadata(
        accession_ids=ids,
        email=email,
        n_jobs=n_jobs,
        log_level='WARNING'
    )
    marine_meta_df = marine_meta.view(pd.DataFrame)
    marine_meta_df.to_csv(marine_meta_path, sep='\t', header=True, index=True)
else:
    marine_meta_df = pd.read_csv(marine_meta_path, sep='\t', index_col=0)
    marine_meta_df['Public'] = marine_meta_df['Public'].astype(str)
    print(f'Metadata was read from file "{marine_meta_path}"')
    
print(f'Metadata table contains {marine_meta_df.shape[0]} samples and {marine_meta_df.shape[1]} columns.')    

## Investigate and clean up the metadata
We will look what columns are available in the metadata table and prepare it for the following analysis.

In [ ]:
marine_meta_df.head()

In [ ]:
marine_meta_df.columns

As we want to plot the geographical location of all the samples, we will clean up the required columns.

In [ ]:
# remove NaN values
marine_meta_df = marine_meta_df[
    marine_meta_df['Latitude start [sample]'].notna() & marine_meta_df['Longitude start [sample]'].notna()
]
marine_meta_df.shape

In [ ]:
# ensure correct data type
for col in ['Latitude start [sample]', 'Longitude start [sample]']:
    marine_meta_df[col] = pd.to_numeric(marine_meta_df[col])

In [ ]:
# ensure correct value range
marine_meta_df = marine_meta_df[
    (-90 <= marine_meta_df['Latitude start [sample]']) &  (marine_meta_df['Latitude start [sample]'] <= 90) & \
    (-180 <= marine_meta_df['Longitude start [sample]']) &  (marine_meta_df['Longitude start [sample]'] <= 180)
]
marine_meta_df.shape

We will be exploring some other metadata columns - let's clean those up as well.

In [ ]:
# ensure correct value ranges
columns = (
    'Temperature [sample]', 'Depth [sample]', 'Salinity sensor [sample]', 
    'Oxygen sensor [sample]', 'Nitrate sensor [sample]', 'Chlorophyll sensor [sample]'
)
max_vals = (100, 1000, 100, 500, 100, 10)

for col, max_val in zip(columns, max_vals):
    marine_meta_df = clean_up_column(marine_meta_df, col, max_val)
    print(col, marine_meta_df.shape)

We can also parse the date columns into the datetime format (and convert to string, as required by QIIME 2):

In [ ]:
for col in ('Event date/time start [sample]', 'Event date/time end [sample]'):
    marine_meta_df[col] = pd.to_datetime(marine_meta_df[col], utc=True)
    marine_meta_df[col] = marine_meta_df[col].dt.strftime('%Y-%m-%d')

Finally, we save cleaned metadata to file.

In [ ]:
if not os.path.isfile(marine_meta_clean_path):
    marine_meta_df.to_csv(marine_meta_clean_path, sep='\t', header=True, index=True)

## Summarize samples
The samples we fetched belong to several BioProjects which correspond to fractions obtained through size fractionation used to separate viruses, bacteria and protists from one another. Let's first see how many samples we have per project:

In [ ]:
sample_counts = marine_meta_df['Bioproject ID'].value_counts()

sns.set(rc={'figure.figsize':(12, 5), 'font.family': ['Arial']}, style='white')
with sns.plotting_context("notebook", font_scale=1.2):
    ax = sns.barplot(x=sample_counts.index, y=sample_counts.values, color='dodgerblue')

    ax.set_xlabel('BioProject ID')
    ax.set_ylabel('Sample count')
    
    sns.despine(left=True)

We can also visualize size range for each fraction using the lower and upper limits provided in the metadata table.

In [ ]:
replace_dict = {'<': 0, '>': 10000}

fractions = marine_meta_df.loc[:,
    ['Size fraction lower threshold [sample]', 'Size fraction upper threshold [sample]', 'Bioproject ID']
].replace(replace_dict)

for col in ('Size fraction lower threshold [sample]', 'Size fraction upper threshold [sample]'):
    fractions[col] = fractions[col].astype(float)

In [ ]:
fractions = fractions \
    .groupby(['Size fraction lower threshold [sample]', 'Size fraction upper threshold [sample]']) \
    .count()['Bioproject ID'] \
    .unstack()

In [ ]:
sns.set(rc={'figure.figsize':(10, 8), 'font.family': ['Arial']}, style='white')
with sns.plotting_context("notebook", font_scale=1.4):
    ax = sns.heatmap(data=fractions)

    ax.set_xlabel('Fraction upper threshold [μm]')
    ax.set_ylabel('Fraction lower threshold [μm]')
    
    ax.invert_yaxis()
    
    xlabels = ax.get_xticklabels()
    xlabels[-1] = '>'
    ylabels = ax.get_yticklabels()
    ylabels[0] = '<'
    ax.set_yticklabels(ylabels, rotation=0)
    ax.set_xticklabels(xlabels, rotation=0)

In [ ]:
# switch off inline mode to correctly display QIIME 2 visualizations
%matplotlib auto

## Visualize selected metadata using samples' geographic coordinates
We can use the [**q2-coordinates**](https://github.com/bokulich-lab/q2-coordinates) plugin to visualize a variety of sensor values (and other metadata) according to their geographical locations.

In [ ]:
n_samples = 100
latitude_col = 'Latitude start [sample]'
longitude_col = 'Longitude start [sample]'

marine_smp = marine_meta_df.sample(n=n_samples, random_state=random_seed)
marine_smp.shape

### TARA Oceans expedition progress
We will use the sampling date as a categorical variable to display progress of the expedition.

In [ ]:
sorted_samples = marine_smp.sort_values(by='Event date/time start [sample]', ascending=False)

progress_map, = coordinates.visualizers.draw_interactive_map(
    metadata=q2.Metadata(sorted_samples),
    latitude=latitude_col,
    longitude=longitude_col,
    column='Event date/time start [sample]',
    color_palette='magma'
)
progress_map

### Temperature

In [ ]:
temp_map, = coordinates.visualizers.draw_interactive_map(
    metadata=q2.Metadata(marine_smp),
    latitude=latitude_col,
    longitude=longitude_col,
    column='Temperature [sample]'
)
temp_map

### Depth

In [ ]:
depth_map, = coordinates.visualizers.draw_interactive_map(
    metadata=q2.Metadata(marine_smp),
    latitude=latitude_col,
    longitude=longitude_col,
    column='Depth [sample]',
    color_palette='viridis'
)
depth_map

### Salinity

In [ ]:
salinity_map, = coordinates.visualizers.draw_interactive_map(
    metadata=q2.Metadata(marine_smp),
    latitude=latitude_col,
    longitude=longitude_col,
    column='Salinity sensor [sample]'
)
salinity_map

### Oxygen content

In [ ]:
oxygen_map, = coordinates.visualizers.draw_interactive_map(
    metadata=q2.Metadata(marine_smp),
    latitude=latitude_col,
    longitude=longitude_col,
    column='Oxygen sensor [sample]',
    color_palette='viridis'
)
oxygen_map

### Nitrate content

In [ ]:
nitrate_map, = coordinates.visualizers.draw_interactive_map(
    metadata=q2.Metadata(marine_smp),
    latitude=latitude_col,
    longitude=longitude_col,
    column='Nitrate sensor [sample]',
    color_palette='viridis'
)
nitrate_map

### Chlorophyll content

In [ ]:
chlorophyll_map, = coordinates.visualizers.draw_interactive_map(
    metadata=q2.Metadata(marine_smp),
    latitude=latitude_col,
    longitude=longitude_col,
    column='Chlorophyll sensor [sample]',
    color_palette='terrain'
)
chlorophyll_map

## Fetch a sample of metagenome sequences using q2-fondue
We can now fetch sequences corresponding to some of those 100 samples that we visualized above. We will only focus on samples from two different locations (5 samples per location; 10 samples total) as they correspond to a very large amount of data. We begin by identifying locations that were multiple samples are available.

In [ ]:
lat_col, lon_col = 'Latitude end [sample]', 'Longitude end [sample]'

In [ ]:
# find all locations with exactly 5 samples
loc_counts = marine_meta_df.groupby([lat_col, lon_col]).count()
loc_counts_multi = loc_counts[loc_counts['Experiment ID'] == 5]
loc_counts_multi.shape

In [ ]:
# draw two locations of the previous list
loc_smp = loc_counts_multi.sample(n=2, random_state=random_seed)
lats = loc_smp.index.get_level_values(lat_col).tolist()
lons = loc_smp.index.get_level_values(lon_col).tolist()

In [ ]:
marine_loc_smp = marine_meta_df[
    marine_meta_df[lat_col].isin(lats) & \
    marine_meta_df[lon_col].isin(lons)
]
marine_loc_smp.groupby([lat_col, lon_col]).count().iloc[:, :2]

Finally, we are ready to download the actual sequences.

In [ ]:
single_reads_out = os.path.join(results_path, 'tara-single.qza')
paired_reads_out = os.path.join(results_path, 'tara-paired.qza')

In [ ]:
if not os.path.isfile(paired_reads_out):
    _ids = pd.Series(marine_loc_smp.index.to_list(), name='ID')
    _, paired_reads, _ = fondue.methods.get_sequences(
        accession_ids=q2.Artifact.import_data('NCBIAccessionIDs', _ids), 
        email=email,
        n_jobs=n_jobs
    )
    paired_reads.save(paired_reads_out)
else:
    print(f'Paired-reads artifact exists and will be read from {paired_reads_out}.')
    paired_reads = q2.Artifact.load(paired_reads_out)

## Subsample the reads
We will subsample the reads in every sample to 20% of the original count to reduce the amount of data. The final sample will become an artifact containing only single-end reads that were drawn from both, forward and reverse, reads in the paired-end artifact.

In [ ]:
single_reads_smp_out = os.path.join(results_path, 'single-reads-smp.qza')
if not os.path.isfile(single_reads_smp_out):
    single_reads_smp, = demux.methods.subsample_single(
        sequences=paired_reads,
        fraction=0.2
    )
    single_reads_smp.save(single_reads_smp_out)
else:
    print(f'Subsampled single-reads artifact exists and will be read from {single_reads_smp_out}.')
    single_reads_smp = q2.Artifact.load(single_reads_smp_out)

## Calculate and compare MinHash signatures for every metagenome
We will calculate MinHash signatures of every sample (metagenome) using `q2-sourmash`. First, we calculate the hashes from the short reads using the `compute` action. Subsequently, we generate a distance matrix comparing hashes pairwise (using the `compare` action).

In [ ]:
genome_hash_out = os.path.join(results_path, 'genome-hash.qza')
if not os.path.isfile(genome_hash_out):
    genome_hash, = sourmash.methods.compute(
        sequence_file=single_reads_smp,
        ksizes=31,
        scaled=1000
    )
    genome_hash.save(genome_hash_out)
else:
    print(f'Genome hashes artifact exists and will be read from {genome_hash_out}.')
    genome_hash = q2.Artifact.load(genome_hash_out)

In [ ]:
hash_compare_out = os.path.join(results_path, 'hash-compare.qza')
if not os.path.isfile(hash_compare_out):
    hash_compare, = sourmash.methods.compare(
        min_hash_signature=genome_hash,
        ksize=31
    )
    hash_compare.save(hash_compare_out)
else:
    print(f'Distance matrix artifact exists and will be read from {hash_compare_out}.')
    hash_compare = q2.Artifact.load(hash_compare_out)

## Perform dimensionality reduction of the metagenome MinHash distance matrix
Finally, a 2D PCoA plot is generated from the obtained distance matrix (`pcoa` method from the `q2-diversity` plugin) and visualized using the EMPeror plot (`plot` action from the `q2-emperor` plugin).

In [ ]:
genome_pcoa, = diversity.methods.pcoa(
    distance_matrix=hash_compare,
)

In [ ]:
emperor_plot_out = os.path.join(results_path, 'emperor-plot.qzv')
if not os.path.isfile(emperor_plot_out):
    emperor_plot, = emperor.visualizers.plot(
        pcoa=genome_pcoa,
        metadata=q2.Metadata(marine_loc_smp)
    )
    emperor_plot.save(emperor_plot_out)
else:
    print(f'Emperor plot artifact exists and will be read from {emperor_plot_out}.')
    emperor_plot = q2.Visualization.load(emperor_plot_out)

In [ ]:
emperor_plot

We can also use the results above to generate our own plots using any of the Python plotting libraries - see below.

In [ ]:
pcoa_table = genome_pcoa.view(skbio.OrdinationResults)
pcoa_df = pcoa_table.samples

In [ ]:
# switch to inline plotting
%matplotlib inline

In [ ]:
# create a 2D plot of Dim1 vs Dim2

sns.set(rc={'figure.figsize':(8, 8), 'font.family': ['Arial']}, style='white')
with sns.plotting_context("notebook", font_scale=1.2):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.set_xlabel(f'Axis 1 ({round(100 * pcoa_table.proportion_explained[0], 2)}%)')
    ax.set_ylabel(f'Axis 2 ({round(100 * pcoa_table.proportion_explained[1], 2)}%)')
    
    sns.scatterplot(
        x=pcoa_df.iloc[:, 0],
        y=pcoa_df.iloc[:, 1],
        s=200,
        hue=marine_loc_smp['Latitude end [sample]'],
        palette='Dark2',
        ax=ax,
    )

    ax.set_xticks([])
    ax.set_yticks([])
    
    plt.tight_layout()

In [ ]:
fig.savefig(os.path.join(results_path, 'tara_pcoa.eps'))

We can see in the plots above that the samples obtained in the two analyzed locations can be separated into distinct groups using only their MinHash signatures.